<a href="https://colab.research.google.com/github/componavt/sns4human/blob/main/src/vk/nlp/get_posts_tokens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script processes text data, tokenizes it, and saves it to a CSV file.
It filters out entries with empty 'tokens' fields and ensures 'tokens' are always quoted, while 'date' remains unquoted.

Этот скрипт обрабатывает текстовые данные, токенизирует их и сохраняет в CSV-файл.
Он отфильтровывает записи с пустыми полями «токены» и гарантирует, что «токены» всегда заключены в кавычки, а «дата» остается без кавычек.

In [16]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
!pip install -U pymorphy3
import pymorphy3
import requests
import csv

nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)

from nltk.corpus import stopwords
stop_words = stopwords.words("russian")
stop_words += requests.get('https://raw.githubusercontent.com/componavt/sns4human/refs/heads/main/src/vk/nlp/RussianStopWords.txt').text.split('\n')
stop_words += requests.get('https://raw.githubusercontent.com/componavt/sns4human/refs/heads/main/src/vk/nlp/stopwords-ru.txt').text.split()
alphabet = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя-')
morph = pymorphy3.MorphAnalyzer(lang='ru')

In [17]:
domains = ['club221681617','concerto','club151359929','pravoslav_karelia']# smallest groups for tests
df = pd.concat([pd.read_csv('https://raw.githubusercontent.com/componavt/sns4human/refs/heads/main/data/vk/posts/religion/' + domain +'.csv',
                            usecols = ['text','date']) for domain in domains], ignore_index=True)
df = df[df['text'].notna() & (df['text'].apply(lambda x: isinstance(x, str)))]
df = df.reset_index()
df = df.drop(columns=['index'])

def get_text_window(words, index, window_size=3):
    """Returns a context window of words around the given index."""
    start = max(0, index - window_size)
    end = min(len(words), index + window_size + 1)
    return ' '.join(words[start:end])


def process_text(text):
    check_hash = False
    processed_parts = []
    words = nltk.word_tokenize(text)
    for i, w in enumerate(words):
      if len(w) == 1:
        continue
      if w == '#':
          check_hash = True
          continue
      if check_hash:
          check_hash = False
          continue
      w_tag = morph.parse(w.strip())[0].tag
      if 'Surn' in w_tag or 'Name' in w_tag or 'Patr' in w_tag:
        context = get_text_window(words, i)
        print(f"Filtered name/surname: {w} | Context: {context}")  # Debug output for context
        continue
      if set(w.lower()).issubset(alphabet):
        if w.isalpha() and w.lower():
          if w.isupper() and len(w) <= 3:
              processed_parts.append(w)
          else:
              res = morph.parse(w.lower())[0].normal_form
              if res not in stop_words:
                  processed_parts.append(res)
    result = ' '.join(processed_parts)
    return str(result)


df['tokens'] = df['text'].apply(lambda x: process_text(x))
df_tokens = df[['tokens', 'date']]

# Removing lines with empty 'tokens'
df_tokens = df_tokens[df_tokens['tokens'].str.strip().astype(bool)]

# Save with quotes only for the "tokens" field
df_tokens.to_csv('tokens.csv', index=False, sep=';', quoting=csv.QUOTE_NONNUMERIC)


Filtered name/surname: Т.Ушаковой | Context: фото из архива Т.Ушаковой или Фрейндлингов )
Filtered name/surname: о.Александр | Context: о.Александр Романушко - священник
Filtered name/surname: Александру | Context: , когда отцу Александру пришлось отпевать одного
Filtered name/surname: Василию | Context: командира партизан генерал-майора Василию Коржа [ 7
Filtered name/surname: Александру | Context: Корж предоставил отцу Александру право принимать решение
Filtered name/surname: Александр | Context: . Однако отец Александр не ограничился этим
Filtered name/surname: Александра | Context: Потрясённые речью отца Александра , полицейские даже
Filtered name/surname: Василий | Context: 10 ] . Василий Корж позже объявил
Filtered name/surname: //ru.wikipedia.org/wiki/Романушко | Context: . https : //ru.wikipedia.org/wiki/Романушко , _Александр_Фёдорович
Filtered name/surname: _Александр_Фёдорович | Context: : //ru.wikipedia.org/wiki/Романушко , _Александр_Фёдорович
Filtered name/surname: Савель